## Энергетический оракул
Ноутбук команды #12

Работа выполнена на основе модели LightGBM


### 1. Подготовка данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

import re

from tqdm import tqdm

random_state = 12345
NUM_ITERATIONS = 5000

In [ ]:
def mae_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)

#### 1.1 Функции для расшифровки прогноза погоды в колонке 'weather_pred'

In [ ]:
# Расшифровка прогноза в колонке 'weather_pred'

# функция формирует колонки 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability', 'has_rain_probability'
# в колонках число, которое 0 при отсутсвии упоминания явления в weather_pred или степень упоминания
# функция дает в колонках номер первого списка, элемент которого есть в строке плюс 1
# списки cloudy_list, rainy_list, windy_list, clear_list можно модифицировать
# соответственно, можно экспериментировать с расположением значений в списках
# например, сейчас 'дождь', 'снег', 'д+сн' - первая степень  дождя, а 'гроз', 'ливень' - вторая
# а можно сделать снег второй, а грозу с ливнем убрать в третью
# также сделал отдельный список для "ясности", чтобы выделить 'ясно' и 'солнечно'

def in_what_list(weather, big_list):
    for list_number, small_list in enumerate(big_list):
        if any(word in weather for word in small_list):
            return list_number+1
    return 0

def weather_split2(row):
    weather = row['weather_pred']
    cloudy_list = [['проясн', 'пер.об.', 'п/об'], ['пасм', 'обл']]
    rainy_list = [['дождь', 'снег', 'д+сн'], ['гроз', 'ливень']]
    windy_list = [['вет'],['штор']]
    clear_list = [['проясн'], ['ясно'], ['солнеч']]
    numbers = re.findall(r'\d+', weather)
    cloudy = in_what_list(weather, cloudy_list)
    rainy = in_what_list(weather, rainy_list)
    windy = in_what_list(weather, windy_list)
    clear = in_what_list(weather, clear_list)
    rain_probability = 0 if len(numbers)==0 else int(numbers[0])
    has_rain_probability = int(len(numbers)==0)
    return cloudy, rainy, windy, clear, rain_probability, has_rain_probability

def fill_weather_columns(df):
    df['weather_pred'] = df['weather_pred'].fillna('')
    df['cloudy'], df['rainy'], df['windy'], df['clear'], df['rain_probability'], df['has_rain_probability'] = \
                zip(*df.apply(weather_split2, axis=1))
    return df

#### 1.2 Функции для загрузки данных о ВВП 
данные загружаются из файла 'data/VVP.csv'

Некоторые научные работы указывают на прямую связь величины потребления электричества и показателя ВВП, который отражает ситуацию в экономике. Данные по экономике публикуются различными министерствами с разной периодичностью. Для использования в работе были взяты фактические данные по ВВП с сайта investing, который агрегирует публикации Минэкономразвития. Данные за месяц побликуются с месячной задержкой, поэтому модель использует для прогнозирования данные за прошлые месяцы, которые известны.   
  
Ссылка на данные: https://ru.investing.com/economic-calendar/russian-monthly-gdp-407


In [ ]:
# Функция добавляет данные о ВВП из файла 'data/VVP.csv' в датасет

def add_vvp2(data, file_source = 'data/VVP.csv'):
    """
    сырой датафрем подаем на вход
    """
    # обработаем файл с динамикой ВВП
    vvp = pd.read_csv(file_source)
    # преобразуем дату файла-источника в формат datetime64 и дропнем один столбик
    vvp['date'] = pd.to_datetime(vvp['date'], format ='%Y-%m-%d %H:%M:%S')
    vvp.drop('for_month',axis=1,inplace=True) 
    
    # обработаем основной фрейм - создадим столбец для соединения, который потом удалим
    data['date_temp'] = pd.to_datetime(data['date'], format = '%Y-%m-%d' )
    data['date_temp'] = data['date_temp'] + pd.to_timedelta(data['time'] , 'H')
    
    # соединяем основной фрейм и ВВП по дате объявления показтеля ВВП
    for idx in reversed(vvp.index):
        data.loc[data['date_temp']>=vvp.date[idx],'VVP'] = vvp.VVP_perc[idx]
        
    data.drop('date_temp',axis=1,inplace=True)   

    return data

#### 1.3 Функции для загрузки архива данных о фактической погоде
данные загружаются из файла 'data/preprocessing_loaded_table.csv'

Изначально данные для формирования таблицы "preprocessing_loaded_table" были взяты из с сайта [https://rp5.ru](https://rp5.ru/Архив_погоды_в_Храброво,_им._императрицы_Елизаветы_Петровны_(аэропорт),_METAR), где хранятся архивы погоды в аэрапорту Калининграда, за период с 31.12.2018 по 30.09.2023

Описание данных в таблице:
- Местное время в Храброво / им. императрицы Елизаветы Петровны (аэропорт) - Дата / Местное время
- T -  Темпиратура воздуха
- Po - Давление на уровне станции
- P - Давление приведённое к уровню моря
- U - Относительная влажность
- DD - Направление ветра
- Ff - Скорость ветра
- ff10 - Максимальное значение порыва ветра
- WW - Особое явление текущей погоды (осадки)
- W'W' - Явление недавней погоды, имеющее оперативное значение
- с - Общая облачность
- VV - Горизонтальная дальность видимости
- Td - Темпиратура точки росы

Данные, которые были взяты из данной таблицы и загружаются из 'data/preprocessing_loaded_table.csv':
- P - не подверглось изменению
- U - не подверглось изменению
- Td - не подверглась изменению

 WW - разделили на 4 категории:
- Нет осадков (где были пропуски)
- слабый дождь
- сильный дождь
- снег

DD - создали 4 столбца, соответствующих сторонам горизонта, которые принимали значения 0; 0.5 и 1 в зависимости от силы ветра в конкретном направлении
- N - north
- S - south
- W - west
- E - east

В дальнейшем эти данные использовались с лагом в сутки: в поля на завтрашний день записывались данные сегодняшнего.

In [ ]:
# Функции для работы с данными о фактической погоде из 'data/preprocessing_loaded_table.csv'

# Кодировка информации об осадках из колонки WW
def true_weather_WW_replace(ww):
    if ww=='нет осадков':
        return 0
    elif ww=='слабый дождь':
        return 1
    elif (ww=='сильный дождь') or (ww=='снег'):
        return 2
    else:
        return 3

# Вычисление Timestamp из даты и времени
def row_plus_hours_to_index(row):
    return row['date'] + pd.to_timedelta(row['time'] , 'H')

# Функция для сдвига на сутки (в скачанном датасете разбивка по 30 мин, поэтому timeshift=48)
def shift_features_fact(df, timeshift=48):
    list_fact_columns=list(df.columns)
    list_fact_columns.remove('date_tw')
    new_df = df.copy()
    for column in list_fact_columns:
        new_df[column] = new_df[column].shift(timeshift)

    return new_df

In [ ]:
# Функция для вычисления метрики mae по дням из почасовых массивов данных

def mae_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)
# Функция для вычисления метрик по дням из почасовых массивов данных

def metrics_hour(y_true, y_pred):

    
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

#### 1.5 Чтение файлов с данными
Данные объединяются в один датасет

In [ ]:
# читаем исходные датасеты и складываем в один
train_ds = pd.read_csv('data/train_dataset.csv')
test_ds = pd.read_csv('data/test_dataset.csv')
train_ds = pd.concat([train_ds, test_ds])

# запоминаем дату начала тестовых данных, потом также поступим и с закрытым датасетом
open_test_begin = pd.to_datetime(test_ds['date']).min()
open_test_end = pd.to_datetime(test_ds['date']).max() + pd.to_timedelta(1,'d')
print('начало открытого теста:', open_test_begin, '    конец открытого теста:', open_test_end)

In [ ]:
duplicates = train_ds.duplicated()
duplicates.sum()

#### 1.6 Формирование колонок с производными от даты

In [ ]:
# преобразуем дату и делаем из нее колонки
train_ds['date'] = pd.to_datetime(train_ds['date'])
train_ds['year'] = train_ds['date'].dt.year
train_ds['month'] = train_ds['date'].dt.month
train_ds['day_of_week'] = train_ds['date'].dt.dayofweek
train_ds['day'] = train_ds['date'].dt.day
train_ds['day_of_year'] = train_ds['date'].dt.dayofyear

#### 1.7 Подгрузка данных о праздниках

In [ ]:
# Добавление данных о праздниках из файла 'data/holidays.csv'

df_holidays = pd.read_csv('data/holidays.csv')
df_holidays['date'] = pd.to_datetime(df_holidays['date'])

# Assuming df_holidays and train_ds are your dataframes
train_ds = pd.merge(train_ds, df_holidays, on='date', how='left')
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())
# Fill NaN values with 0
train_ds['holidays'].fillna(0, inplace=True)
train_ds['preholidays'].fillna(0, inplace=True)

# Convert to int
train_ds['holidays'] = train_ds['holidays'].astype(int)
train_ds['preholidays'] = train_ds['preholidays'].astype(int)

#### 1.8 Формирование колонок со значением целевого признака в предыдущие дни

In [ ]:
# Добавление колонок с временными лагами

# создаем столбец 'temp_last_day'
train_ds['temp_last_day'] = train_ds['temp'].shift(24)

# заполняем пропущенные значения в 'temp_last_day'
train_ds['temp_last_day'].fillna(method='bfill', inplace=True)

# создаем столбцы с временными лагами для 'target'
lags = [1, 12,  24, 35, 40, 48, 72, 7*24, 14*24]
for lag in lags:
    train_ds[f'target_lag_{lag}'] = train_ds['target'].shift(lag)

"""count_rollins = [24,
                    #7*24, 
                    #28*24,
                    #56*24
                    ] 
for count_rollin in count_rollins:  
    train_ds[f'mean_{count_rollin}'] = train_ds['target_lag_24'].rolling(count_rollin).mean()

count_rollins = [24, 7*24, 28*24] 
for count_rollin in count_rollins:  
    train_ds[f'temp_mean_{count_rollin}'] = train_ds['temp'].rolling(count_rollin).mean().shift(24)"""
# заполняем пропущенные значения в столбцах с лагами
for lag in lags:
    train_ds[f'target_lag_{lag}'].fillna(0, inplace=True)

In [ ]:
duplicates = train_ds.duplicated()
duplicates.sum()

#### 1.9 Формирование колонок с ВВП и данными о погоде посредством ранее описанных функций

In [ ]:
# применяем функцию добавления ВВП
train_ds = add_vvp2(train_ds)

# Расшифровка прогноза в колонке 'weather_pred'
train_ds = fill_weather_columns(train_ds)


# Читаем файл с архивом фактической погоды
df_true_weather = pd.read_csv('data/preprocessing_loaded_table.csv')
display(df_true_weather)

# Форматируем колонки
df_true_weather['WW'] = df_true_weather['WW'].apply(true_weather_WW_replace)
df_true_weather['date'] = pd.to_datetime(df_true_weather['date'])
df_true_weather = df_true_weather.rename(columns={'date':'date_tw'})
# Применяем сдвиг на сутки, чтобы не заглядывать в будущее
df_true_weather = shift_features_fact(df_true_weather)
# Добавляем в датасет
train_ds['date_hours'] = train_ds.apply(row_plus_hours_to_index, axis=1)
train_ds = train_ds.merge(df_true_weather, left_on='date_hours', right_on='date_tw')
train_ds = train_ds.drop(['date_hours', 'date_tw'], axis=1)

In [ ]:
duplicates = train_ds.duplicated()
duplicates.sum()

#### 1.10 Демонстрация сформированного датасета

In [ ]:
# Итоговый набор колонок
train_ds.columns

In [ ]:
train_ds.head()

In [ ]:
duplicates = train_ds.duplicated()
duplicates.sum()

#### 1.11 Исключение лишних колонок

In [ ]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 

feature_cols = list(train_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'day_of_year', 'weather_pred', 'weather_fact', 'temp']

# выбрасываем признаки, найденные процедурно в процессе оптимизации
# КОМАНДЕ: здесь можно добавлять признаки на выброс с целью оптимизации
drop_list = drop_list + ['target_lag_48', 'target_lag_168', 
                         'has_rain_probability', 'preholidays',
                          'W', 'E'] 

for name in drop_list:
    feature_cols.remove(name)

# Итоговый список признаков
feature_cols

#### 1.12 Выделение наборов данных для обучения, валидации и тестирования

Выделялось два набора данных для обучения и валидации:
1. Обучение на данных с 2019 по 2021 с валидацией на 2022
2. Обучение на данных с 2019 по 2022 с валидацией на первом квартале 2023

Первый набор позволяет оценить влияние сезонности на обучение и предсказания, второй позволяет обучить модель на большем объеме данных и на более актуальных данных.

In [ ]:
# Формируем набор датасетов для обучения и проверки

features = train_ds[feature_cols]
target = train_ds['target']

# Функция для выделения временных интервалов из таблиц признаков и целей
# на этом этапе отбрасываем колонку 'date'
def features_interval(features, target, date1, date2):
    features_interval = features[ (features['date']>=date1) & (features['date']<date2) ]
    target_interval = target[features_interval.index]
    features_interval = features_interval.drop('date', axis=1)
    return features_interval, target_interval



# отбор признаков будем производить, обучая на 19-22 и проверяя по первому кварталу 2023
# с дополнительным контролем на вариантах из первичного обучения
features_2022, target_2022 = features_interval(features, target, '2019-01-01', '2023-01-01')
features_2023, target_2023 = features_interval(features, target, '2023-01-01', open_test_begin)

# для проверки на тестовой выборке будем учиться на всем тренировочном датасете
features_all_train, target_all_train = features_interval(features, target, '2019-01-01', open_test_begin)
features_open_test, target_open_test = features_interval(features, target, open_test_begin, open_test_end)



In [ ]:
duplicates = features.duplicated()
duplicates.sum()

### 2. Обучение моделей

В настоящей работе обучается модель LightGBM

#### 2.1 Гиперпараметры
Были подобраны следующие значения гиперпараметров:

In [ ]:
params = {'num_leaves':15, 'learning_rate':0.02, 'feature_fraction':1, 'num_iterations':NUM_ITERATIONS, 'random_state':random_state, 'objective':'regression_l1', 'n_jobs':-1}

#### 2.3 Процедурный отбор признаков
Для каждого признака проверим метрику без него и сравним с базовой метрикой, полученной со всеми признаками. Данная процедура позволит улучшить результаты за счет исключения признаков. Здесь обучение проходит на данных за 2019-2022 годы с валидацией на первом квартале 2023.

In [ ]:
features_all_train.info()

### 4 Проверка метрик на тестовом датасете

In [ ]:

feature_lgbm =  features_all_train.drop(columns=['target_lag_1','target_lag_12'])
feature_lgbm_test = features_open_test.drop(columns=['target_lag_1','target_lag_12'])

In [ ]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете
params = {'num_leaves':15, 'learning_rate':0.02, 'feature_fraction':0.98, 'num_iterations':10000, 'random_state':random_state, 'objective':'regression_l1', 'n_jobs':-1}

lgbm_model_all_train = lgb.LGBMRegressor(**params)
lgbm_model_all_train.fit(feature_lgbm, target_all_train)



In [ ]:
predict_train = lgbm_model_all_train.predict(feature_lgbm)
predict_test = lgbm_model_all_train.predict(feature_lgbm_test)

In [ ]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train,predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test,predict_test )

pd.DataFrame([['тренировочная на часе', mae_train, mape_train, r2_train], ['тестовая', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка на часе', 'MAE', 'MAPE', 'R2'))

In [ ]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете
params = {'num_leaves':15, 'learning_rate':0.02, 'feature_fraction':0.98, 'num_iterations':10000, 'random_state':random_state, 'objective':'regression_l1', 'n_jobs':-1}

lgbm_model_lag = lgb.LGBMRegressor(**params)
lgbm_model_lag.fit(features_all_train, target_all_train)



In [ ]:
predict_test_df = []
predictions = [0]*24  # инициализируем список для хранения последних 24 предсказаний

for i in range(len(features_open_test)):
    row = features_open_test.iloc[i]
    row_time = int(row['time'])

    if row['time'] > 0:
        row['target_lag_1'] = predictions[row_time - 1]
    if row['time'] > 11:
        row['target_lag_12'] = predictions[row_time - 12]
    #if row['time'] > 22:
    #    row['target_lag_23'] = predictions[row_time - 23]

    prediction = lgbm_model_lag.predict(row.values.reshape(1, -1))[0]
    predict_test_df.append(prediction)
    
    # обновляем список предсказаний
    predictions[row_time] = prediction

predict_train_df = []
predictions = [0]*24  # инициализируем список для хранения последних 24 предсказаний

for i in range(len(features_all_train)):
    row = features_all_train.iloc[i]
    row_time = int(row['time'])
    
    if row['time'] > 0:
        row['target_lag_1'] = predictions[row_time - 1]
    if row['time'] > 11:
        row['target_lag_12'] = predictions[row_time - 12]
    #if row['time'] > 22:
    #    row['target_lag_23'] = predictions[row_time - 23]

    prediction = lgbm_model_lag.predict(row.values.reshape(1, -1))[0]
    predict_train_df.append(prediction)
    
    # обновляем список предсказаний
    predictions[row_time] = prediction



In [ ]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train,predict_train_df )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test,predict_test_df )

pd.DataFrame([['тренировочная на часе', mae_train, mape_train, r2_train], ['тестовая', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка на часе', 'MAE', 'MAPE', 'R2'))

In [ ]:
print(f'Mae day = {mae_day(target_open_test,predict_test_df)}')

#### 2.4 График важности признаков
Визуализируем значение feature_importances_ модели

In [ ]:
# График важности признаков

tmp_feature_cols = feature_cols.copy()
tmp_feature_cols.remove('date')
tmp_feature_cols.remove('target_lag_1')
feature_importances = pd.DataFrame([tmp_feature_cols, lgbm_model_all_train.feature_importances_]).T.sort_values(by = 1)
feature_importances.plot(kind='barh', x=0, y=1, figsize=(18, 8))

In [ ]:
# Сохранение модели
lgbm_model_all_train.booster_.save_model('lgbm_model_pred.txt')

In [ ]:
features_open_test.head(3)

### Optuna search

In [ ]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor

import optuna
from optuna.samplers import TPESampler

In [ ]:
params = {'num_leaves':15, 'learning_rate':0.02, 'feature_fraction':0.98, 
'num_iterations':10000, 'random_state':random_state, 'objective':'regression_l1', 'n_jobs':-1}

In [ ]:
X_train, y_train = feature_lgbm, target_all_train

In [ ]:
X_train.head(3)

In [ ]:
def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01,  0.1),
        'objective': trial.suggest_categorical('objective', ['regression_l1', 'regression_l2']),
        'random_state':random_state,
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'n_jobs': 10,
        'num_iterations':10000      
                }
    
    model = LGBMRegressor( **params)
    
    tscv = TimeSeriesSplit(n_splits=3)
    
    scores = []
    for train_index, valid_index in tqdm(tscv.split(X_train), total=3):
        X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[valid_index]
        y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[valid_index]
        
        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_valid_fold)
        
        score = mean_absolute_error(y_valid_fold, y_pred)
        scores.append(score)
    
    return np.mean(scores)


In [ ]:
study = optuna.create_study(direction='minimize', sampler=TPESampler())
study.optimize(objective, n_trials=100)

print(study.best_params)

In [ ]:
print(study.best_params)

In [ ]:
params= {'num_leaves': 34, 'min_child_samples': 16, 
          'max_depth': 8, 'learning_rate': 0.01206843641003463, 
          'objective': 'regression_l1', 'feature_fraction': 0.9574152630927155,
          'n_jobs':-1, 'num_iterations':10000
          }


In [ ]:
print(study.best_trial.value)

In [ ]:
study= [FrozenTrial(number=73, state=1, values=[9.761631678943006],
 datetime_start=datetime.datetime(2023, 11, 2, 21, 58, 0, 233317), 
 datetime_complete=datetime.datetime(2023, 11, 2, 22, 5, 15, 146635), 
 params={'num_leaves': 34, 'min_child_samples': 16, 'max_depth': 8, 
 'learning_rate': 0.01206843641003463, 'objective': 'regression_l1', 
 'feature_fraction': 0.9574152630927155}, user_attrs={}, system_attrs={},
  intermediate_values={}, distributions={'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 
  'min_child_samples': IntDistribution(high=200, log=False, low=3, step=1),
   'max_depth': IntDistribution(high=8, log=False, low=3, step=1), 
   'learning_rate': FloatDistribution(high=0.1, log=True, low=0.01, step=None),
    'objective': CategoricalDistribution(choices=('regression_l1', 'regression_l2')),
     'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.1, step=None)}, trial_id=73, value=None)]


In [ ]:
print(study.best_trial.value)

In [ ]:
model = LGBMRegressor( **params)
model.fit(X_train, y_train)

In [ ]:
predict_train = model.predict(feature_lgbm)
predict_test = model.predict(feature_lgbm_test)

In [ ]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train,predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test,predict_test )

pd.DataFrame([['тренировочная на часе', mae_train, mape_train, r2_train], ['тестовая', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка на часе', 'MAE', 'MAPE', 'R2'))